In [6]:
import pandas as pd
import numpy as np
from bertopic import BERTopic
import sqlite3
import sqllite_handler
import copy

### transfer below block into analysis.py (ALREADY DONE, DELETE BELOW)

In [ ]:
conn = sqlite3.connect('tables/texts_69_76.db')
cur = conn.cursor()

res = cur.execute("SELECT TEXT FROM transcript")
fetched = res.fetchall()
free_text_list = list(map(lambda x: x[0], fetched))

topic_model = BERTopic.load("plots/topic_model_69_76")

doc_df = pd.read_csv('tables/doc_69_76.csv')

messy_doc_topic_df = topic_model.get_document_info(free_text_list)

topic_desc_df = messy_doc_topic_df[['Name','Top_n_words']].drop_duplicates(ignore_index=True)
doc_topic_df = pd.DataFrame({'id_to_text':doc_df['id_to_text'],'assigned_topic':messy_doc_topic_df['Name']})

topic_desc_df.to_csv('tables/topic_descp_69_76.csv')
doc_topic_df.to_csv('tables/doc_topic_69_76.csv')

### construction below

### LDA and Dynamic LDA

In [3]:
# LDA topic modeling

import gensim
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
from gensim.models.ldaseqmodel import LdaSeqModel
import spacy
import pickle
import math
import ray
ray.init(num_cpus=12)

pyLDAvis.enable_notebook()
nlp = spacy.load('en_core_web_sm')

/Users/gokberk/miniconda3/envs/ml4hc_project2/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
/Users/gokberk/miniconda3/envs/ml4hc_project2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/gokberk/miniconda3/envs/ml4hc_project2/lib/python3.9/site-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/Users/gokberk/miniconda3/envs/ml4hc_project2/lib/python3.9/site-packages/spacy/util.py:87

In [7]:
doc_df = pd.read_csv('tables/tables_69_76/doc_69_76v30.csv')
doc_df = doc_df[doc_df['subtype']!='editorial-note']
free_text_list = doc_df['text'].values
free_text_list = list(map(lambda x: ' ' if not x or (isinstance(x, float) and math.isnan(x)) else x, free_text_list))

In [8]:
@ray.remote
def preprocess(row):
    return [t.lemma_.lower() for t in nlp(row) if not t.is_punct and not t.is_stop and t.ent_iob_=='O' and len(t.text)>=3]
# and t.pos_ in ['PROPN','NOUN']

futures = [preprocess.remote(row) for row in free_text_list]
processed_free_text_list = ray.get(futures)

2023-03-28 13:38:28,014	WARNING worker.py:1257 -- Warning: The remote function __main__.preprocess is very large (16 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.


In [ ]:
'''with open("tables/tables_69_76/lda_processed_entremoved_onlynoun_free_text_list", "wb") as fp:
    pickle.dump(processed_free_text_list, fp)'''

In [8]:
# LDA
dictionary = Dictionary(processed_free_text_list)

dictionary.filter_extremes(keep_n=10000)

doc_term_matrix = [dictionary.doc2bow(doc) for doc in processed_free_text_list]

lda = LdaModel(corpus = doc_term_matrix, id2word=dictionary, num_topics=30)

gensimvis.prepare(lda, doc_term_matrix, dictionary)

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
14    -0.133393  0.050641       1        1  8.229308
19     0.106201 -0.084435       2        1  8.114764
23    -0.017818 -0.022370       3        1  7.025212
12     0.082993 -0.077634       4        1  6.496763
4     -0.111643 -0.010516       5        1  5.429920
27     0.093163  0.114505       6        1  4.275438
22     0.028418 -0.055853       7        1  3.806627
5     -0.085972 -0.007288       8        1  3.644824
3     -0.076683 -0.041571       9        1  3.577717
11     0.207341 -0.055088      10        1  3.572346
26     0.076209  0.088635      11        1  3.551748
25     0.203143 -0.063482      12        1  3.462445
13    -0.048378  0.148193      13        1  3.418569
18    -0.087263 -0.034432      14        1  3.359919
9     -0.136086 -0.032758      15        1  3.310503
29    -0.000078 -0.167652      16        1  2.926372
7      0.062502  0.051892      17        1  2.806008
1     -0.131622 -0.070806      18        1  2.767033
20    -0.104668  0.157035      19        1  2.510799
28     0.076101  0.062168      20        1  2.409257
15     0.014678  0.202070      21        1  2.194850
16     0.045163  0.115459      22        1  2.108845
10     0.103123 -0.034901      23        1  1.979475
0      0.013915 -0.068093      24        1  1.849731
21     0.134939 -0.006660      25        1  1.408765
17    -0.045600 -0.024804      26        1  1.394437
8      0.062048  0.043558      27        1  1.343793
6     -0.175983 -0.039129      28        1  1.338476
24    -0.082773 -0.074465      29        1  1.258290
2     -0.071980 -0.062220      30        1  0.427765, topic_info=          Term          Freq         Total Category  logprob  loglift
186        mr.  50786.000000  50786.000000  Default  30.0000  30.0000
218  secretary  53724.000000  53724.000000  Default  29.0000  29.0000
182   minister  23983.000000  23983.000000  Default  28.0000  28.0000
1    agreement  39137.000000  39137.000000  Default  27.0000  27.0000
924          k   9502.000000   9502.000000  Default  26.0000  26.0000
..         ...           ...           ...      ...      ...      ...
92        view     74.133746  19679.011744  Topic30  -5.4001  -0.1271
252     effort     73.627850  18251.358465  Topic30  -5.4070  -0.0586
15     country     74.177203  34567.392303  Topic30  -5.3995  -0.6899
37       issue     70.020571  30296.494848  Topic30  -5.4572  -0.6156
229    support     68.400466  14305.081668  Topic30  -5.4806   0.1114

[2372 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
5972      8  0.894317     12th
4891      8  0.926799     13th
827       3  0.011811  1970–73
827       8  0.023622  1970–73
827      13  0.791332  1970–73
...     ...       ...      ...
573      26  0.000384     zone
573      27  0.001537     zone
573      28  0.000769     zone
573      29  0.002691     zone
1874     30  0.892284    zones

[17879 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 20, 24, 13, 5, 28, 23, 6, 4, 12, 27, 26, 14, 19, 10, 30, 8, 2, 21, 29, 16, 17, 11, 1, 22, 18, 9, 7, 25, 3])

In [14]:
#DYNAMIC LDA

year_list = list(map(lambda x: int(x),doc_df['year']))
year_text_tuple = list(zip(year_list, processed_free_text_list))
year_text_tuple = sorted(year_text_tuple, key=lambda x: x[0])
sorted_year_list = list(map(lambda x: x[0],year_text_tuple))
sorted_text_list = list(map(lambda x: x[1],year_text_tuple))

In [ ]:
# Gensim's dynamic lda is too slow.
dictionary = Dictionary(sorted_text_list)

dictionary.filter_extremes(keep_n=100)

doc_term_matrix = [dictionary.doc2bow(doc) for doc in sorted_text_list]

time_slice = pd.Series(sorted_year_list).value_counts(sort=False).values

ldaseq = LdaSeqModel(corpus = doc_term_matrix, time_slice=list(time_slice), id2word=dictionary, num_topics=5)

In [27]:
# new library LDA fitting

import tomotopy as tp
import sys

# define model
mdl = tp.LDAModel(k=5)

# add documents
for txt in processed_free_text_list:
    mdl.add_doc(txt)

# train model
for i in range(0, 1000, 10):
    mdl.train(10)

topic_term_dists = np.stack([mdl.get_topic_word_dist(k) for k in range(mdl.k)])
doc_topic_dists = np.stack([doc.get_topic_dist() for doc in mdl.docs])
doc_topic_dists /= doc_topic_dists.sum(axis=1, keepdims=True)
doc_lengths = np.array([len(doc.words) for doc in mdl.docs])
vocab = list(mdl.used_vocabs)
term_frequency = mdl.used_vocab_freq

pyLDAvis.prepare(topic_term_dists, doc_topic_dists, doc_lengths, vocab, term_frequency,start_index=0,sort_topics=False)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.323474  0.115257       0        1  17.398626
1      0.041676 -0.375072       1        1  12.987377
2     -0.109236  0.154963       2        1  20.738568
3      0.035328  0.079045       3        1  22.237352
4     -0.291241  0.025807       4        1  26.638077, topic_info=          Term         Freq        Total Category  logprob  loglift
0    secretary  1787.000000  1787.000000  Default  30.0000  30.0000
3          mr.   783.000000   783.000000  Default  29.0000  29.0000
1       cyprus  1047.000000  1047.000000  Default  28.0000  28.0000
2     military   836.000000   836.000000  Default  27.0000  27.0000
5        think   771.000000   771.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
51     meeting   136.711511   258.943264   Topic4  -5.2871   0.6841
15    minister   159.661404   467.776986   Topic4  -5.1320   0.2479
46      policy   129.726763   270.192402   Topic4  -5.3396   0.5891
7   government   147.687555   609.608073   Topic4  -5.2099  -0.0949
37       point   128.728945   312.590655   Topic4  -5.3473   0.4356

[314 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
55        2  0.080527      accept
55        3  0.918012      accept
38        0  0.147759      action
38        3  0.462551      action
38        4  0.391884      action
...     ...       ...         ...
29        4  0.309220         way
185       3  1.001956     willing
230       0  1.001986  withdrawal
53        2  0.998426         yes
361       3  1.001584        zone

[393 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[0, 1, 2, 3, 4])

In [28]:
# new library dynamic LDA fitting

import tomotopy as tp
import sys

# define model
mdl = tp.DTModel(k=5, t=len(np.unique(sorted_year_list)), tw=tp.TermWeight.ONE)

# add documents
for idx,txt in enumerate(sorted_text_list):
    time_pt = sorted_year_list[idx]-min(sorted_year_list)
    mdl.add_doc(words=txt, timepoint=time_pt)

# train model
mdl.train(1000)

In [32]:
for t in range(mdl.num_timepoints):
    print(mdl.get_topic_words(topic_id=3,timepoint=t,top_n=10))
    print('+++')

[('community', 0.029121089726686478), ('island', 0.0221136212348938), ('violence', 0.01251459401100874), ('cyprus', 0.009521933272480965), ('cypriot', 0.009519968181848526), ('intercommunal', 0.009180083870887756), ('cypriots', 0.007131969556212425), ('independence', 0.004716104827821255), ('police', 0.0037465342320501804), ('nation', 0.0037308295723050833)]
+++
[('rough', 0.012905229814350605), ('incidental', 0.010170548222959042), ('intelligence', 0.010099736973643303), ('wh43529', 0.009641645476222038), ('intra', 0.009039051830768585), ('necessitate', 0.00892449077218771), ('durable', 0.008824620395898819), ('yak', 0.008353336714208126), ('primin', 0.008077332749962807), ('party', 0.008073585107922554)]
+++
[('party', 0.021469581872224808), ('intelligence', 0.015901194885373116), ('opposition', 0.009213073179125786), ('junta', 0.00884441938251257), ('political', 0.007938066497445107), ('crisis', 0.006305653136223555), ('community', 0.005764015484601259), ('minister', 0.0045809671282

### below is for parallel removal of named entities from free texts. (save in original code)

In [1]:
import pandas as pd
import numpy as np
import spacy
import pickle
import math
import ray
ray.init(num_cpus=12)

nlp = spacy.load('en_core_web_sm')

name_extension = '_52_88_entremoved'

/Users/gokberk/miniconda3/envs/ml4hc_project2/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/gokberk/miniconda3/envs/ml4hc_project2/lib/python3.9/site-packages/spacy/util.py:877: UserWarning: [W095] Model 'en_core_web_sm' (3.2.0) was trained with spaCy v3.2 and may not be 100% compatible with the current version (3.5.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [2]:
@ray.remote
def remove_entities(txt):
    document = nlp(txt)

    edited_txt = ""
    for token in document:
        
        if token.ent_iob_=='O':
            if token.whitespace_:
                edited_txt += token.text+ ' '
            else:
                edited_txt += token.text
    
    return edited_txt

In [3]:
doc_df = pd.read_csv('tables/tables_52_88/doc.csv')
free_text_list = doc_df['text'].values
free_text_list = list(map(lambda x: ' ' if not x or (isinstance(x, float) and math.isnan(x)) else x, free_text_list))

futures = [remove_entities.remote(txt) for txt in free_text_list]
free_text_list = ray.get(futures)
print('entities removed from free texts.')

with open("tables/tables_52_88/free_text_list"+name_extension, "wb") as fp:
    pickle.dump(free_text_list, fp)

2023-03-27 13:39:04,684	WARNING worker.py:1257 -- Warning: The remote function __main__.remove_entities is very large (16 MiB). Check that its definition is not implicitly capturing a large array or other object in scope. Tip: use ray.put() to put large objects in the Ray object store.


entities removed from free texts.


### INSTITUTION EXTRACTION